# Import necessary library

In [1]:
import pandas as pd
import spacy

# import data from csv

In [22]:
import os
import platform

def check_os_and_load_data():
    if platform.system() == "Windows":
        # Insert the Windows-specific code here
        print("This is a Windows system. Fucking corporate douche.Nevertheless running Windows-specific code.")
        #variante Linus
        data = pd.read_csv('../data/train/train.csv')
        del data['id']
        #data.head()
        return data
    elif platform.system() == "Linux":
        # Insert the Linux-specific code here
        print("This is a Linux system. Wise choice my friend. Running Linux-specific code.")
        #variante Michael
        data = pd.read_csv('~/ownCloud - Michael Saxer (zhaw.ch)@drive.switch.ch/4 Semester/NLP/workbench/train.csv')
        del data['id']
        #data.head()
        return data
    else:
        print("This is neither a Windows nor a Linux system. your on your own sorry.")
        # Insert code for other operating systems or a generic alternative here

# Example usage
data = check_os_and_load_data()




This is a Linux system. Wise choice my friend. Running Linux-specific code.


In [21]:
data.head(8)



,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0


In [23]:
def fun(x):
    if x.sum() != 0:
        return 1
    return 0

rows = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

data['hate'] = data[rows].apply(lambda x: fun(x), axis=1)

for i in rows:
    del data[i]


In [24]:
data.head(10)

,comment_text,hate
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0
